# Why Fine-tuning Was Not Possible in LSTM
## Training
This project attempts to fine-tune a sentiment classification model built with LSTM, and later repurpose it for a sequence-to-sequence task. However, several architectural and practical issues make this fine-tuning ineffective.

## What This Notebook Does

1. Trains an LSTM-based sentiment classifier using the IMDB dataset.
2. Saves the trained model.
3. Loads the model and attempts to fine-tune it on new data with a different vocabulary size.
4. Tries to reuse the LSTM’s hidden states from the classifier as encoder states in a seq2seq architecture.

In [57]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [58]:
max_len = 200
vocab_size = 10000
embedding_dim = 128
latent_dim = 256

In [60]:
(x_train, y_train), _ = tf.keras.datasets.imdb.load_data(num_words=vocab_size)

In [62]:
len(x_train)
len(y_train)

25000

In [63]:
x_train = x_train[:3000]
y_train = y_train[:3000]

In [64]:
len(x_train)
len(y_train)

5000

In [65]:
x_train = pad_sequences(x_train, maxlen=max_len, padding='post', truncating='post')

In [66]:
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(vocab_size, embedding_dim)(input_layer)
lstm_layer, state_h, state_c = LSTM(latent_dim, return_state=True)(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(state_h)
classification_model = Model(input_layer, output_layer)

In [70]:
classification_model.layers

[<InputLayer name=input_layer_5, built=True>,
 <Embedding name=embedding_3, built=True>,
 <LSTM name=lstm_3, built=True>,
 <Dense name=dense_3, built=True>]

In [71]:
classification_model.summary()

Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_3 (Embedding)         │ (None, 200, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ [(None, 256), (None,   │       394,240 │
│                                 │ 256), (None, 256)]     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,023,493 (19.16 MB)

 Trainable params: 1,674,497 (6.39 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,348,996 (12.78 MB)

In [67]:
classification_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [68]:
classification_model.fit(x_train, y_train, batch_size=64, epochs=1, validation_split=0.1)

71/71 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.5011 - loss: 0.6942 - val_accuracy: 0.5180 - val_loss: 0.6926


In [72]:
sample_review = x_train[0].reshape(1, -1)  # 1 sample with shape (1, max_len)
prediction = classification_model.predict(sample_review)

print("Predicted sentiment probability (positive):", prediction[0][0])
print("Predicted Sentiment:", "Positive 😊" if prediction[0][0] > 0.5 else "Negative 😞")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step
Predicted sentiment probability (positive): 0.4902157
Predicted Sentiment: Negative 😞


In [73]:
word_index = imdb.get_word_index()

reverse_word_index = {index + 3: word for word, index in word_index.items()}

reverse_word_index[0] = "<PAD>"
reverse_word_index[1] = "<START>"
reverse_word_index[2] = "<UNK>"
reverse_word_index[3] = "<UNUSED>"
# Step 3: Decode the review
decoded_review = " ".join([reverse_word_index.get(i, "<UNK>") for i in sample_review[0]])

print("****Decoded Review:****")
print(decoded_review)

****Decoded Review:****
<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and s

In [74]:
classification_model.save("lstm_imdb_model.h5")

Here, I take a small subset of 3000 samples and pad the sequences. I define a model like this:

Embedding Layer

LSTM Layer (with return_state=True)

Dense Layer (for binary classification)

I compile the model and train it for 1 epoch. This model outputs a sentiment (positive or negative). I save it as: `lstm_imdb_model.h5`

## Retraininig
#### fine-tune the saved model `lstm_imdb_model.h5`

In [75]:
max_len = 200
vocab_size = 1000
embedding_dim = 128
latent_dim = 256

# Load IMDB dataset
(x_train, y_train), _ = tf.keras.datasets.imdb.load_data(num_words=vocab_size)
x_train = pad_sequences(x_train, maxlen=max_len, padding='post', truncating='post')

In [79]:
x_train = x_train[:1000]
y_train = y_train[:1000]

I load the IMDB data again using this new vocab size. I take a smaller sample of 1000 sequences. I load the previously saved model: `lstm_imdb_model.h5`
Then I attempt to fine-tune this model by calling .fit() again.

In [80]:
load_classification_model = load_model("lstm_imdb_model.h5")

In [81]:
load_classification_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [82]:
load_classification_model.fit(x_train, y_train, batch_size=64, epochs=1, validation_split=0.1)

15/15 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.5386 - loss: 0.6876 - val_accuracy: 0.6000 - val_loss: 0.6835


In [84]:
load_classification_model.save("lstm_imdb_model_updated.h5")

## Retraining for Summarization

In [85]:
updated_classification_model = load_model("lstm_imdb_model_updated.h5")

In [86]:
encoder_inputs = Input(shape=(max_len,))
encoder_embedding = updated_classification_model.layers[1](encoder_inputs)
encoder_outputs, state_h, state_c = updated_classification_model.layers[2](encoder_embedding)

In [87]:
output_vocab_size = 8000
target_max_len = 50

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(output_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])
decoder_dense = Dense(output_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [88]:
seq2seq_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
seq2seq_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [89]:
encoder_input_data = x_train[:1000]  # using IMDB input for now
decoder_input_data = np.random.randint(1, output_vocab_size, (1000, target_max_len))
decoder_target_data = np.random.randint(1, output_vocab_size, (1000, target_max_len, 1))

In [90]:
seq2seq_model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=32,
    epochs=1,
    validation_split=0.1
)


29/29 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - accuracy: 1.2756e-04 - loss: 8.9872 - val_accuracy: 0.0000e+00 - val_loss: 8.9875


## Why Fine-Tuning Fails

### 1. Vocabulary Size Changed

- Initially trained with `vocab_size = 10000`
- Later fine-tuned with `vocab_size = 1000`

The Embedding layer was trained to handle 10,000 tokens. When using only 1,000 tokens during fine-tuning, the token indices refer to **different words**, and the embedding layer now receives **incorrect word meanings**. This leads to corrupted input representations.

---

### 2. Token-to-Word Mapping Is Inconsistent

Even if token IDs are reused, their meanings change when the vocabulary size is reduced. For example:

- In the 10,000-word vocab: token 500 = “great”
- In the 1,000-word vocab: token 500 = “boring”

This breaks the embedding layer, which relies on consistent token mappings.

---

### 3. Using Classifier LSTM State in a Decoder

The model extracts the hidden state from a classifier LSTM:

```python
encoder_outputs, state_h, state_c = classification_model.layers[2](embedding_output)
```

These states are passed into a new decoder LSTM for sequence generation. But this LSTM was **trained to classify sentiment**, not to encode full language meaning. Its hidden states are not meaningful for generation tasks.

---

### 4. No Joint Training of Encoder-Decoder

In proper seq2seq architectures, the encoder and decoder are trained together, so the encoder learns to output useful representations for the decoder. Here, the decoder is randomly initialized and receives context from a classifier encoder — making the setup incompatible.

---

### 5. No Freezing or Layer Control During Fine-Tuning

The model is fine-tuned without freezing layers:

```python
load_classification_model.fit(...)
```

All weights are updated using a new dataset with a different vocabulary and different mappings. This causes **catastrophic forgetting**, where the model loses what it learned in the initial training.

---

## Summary of Why Fine-Tuning Fails

| Problem                         | Why It Breaks Fine-Tuning                                          |
|---------------------------------|---------------------------------------------------------------------|
| Vocabulary size changed         | Embedding layer mismatches the token IDs                           |
| Token mapping changed           | Same IDs now represent different words                             |
| Classification LSTM used as encoder | Hidden states not suitable for generation                        |
| No attention mechanism          | LSTM lacks ability to model long-term dependencies                 |
| No layer freezing               | Entire model updates and forgets previous learning                 |

---

## How to Fix It

To make fine-tuning work properly:

1. Use the **same vocabulary size** across training and fine-tuning (e.g., always `vocab_size = 10000`)
2. Reuse the **same tokenizer and word index**.
3. **Freeze the embedding and LSTM layers** when fine-tuning to prevent forgetting.
4. If using for generation, train the **encoder and decoder together**.
5. Consider using **transformer-based models** (like BERT, GPT, or T5) for more effective transfer learning and modularity.

---

## Conclusion

Fine-tuning is technically possible in LSTMs, but **difficult to get right** due to tight coupling of components, lack of pretraining, and sensitivity to data shifts. Modern architectures like Transformers are better suited for robust fine-tuning and transfer learning.
